In [107]:
from bs4 import BeautifulSoup, SoupStrainer
import requests
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.embeddings import CacheBackedEmbeddings, OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
import requests
from bs4 import BeautifulSoup
from langchain.schema import Document

# 웹 페이지 로드
url = "https://spartacodingclub.kr/blog/all-in-challenge_winner"
response = requests.get(url)
response.encoding = "utf-8"

# HTML 파싱
soup = BeautifulSoup(response.text, "html.parser")


# HTML -> Markdown 변환 함수
def html_to_markdown(soup):
    markdown = ""
    h2s = []

    for element in soup.find_all(True):  # 태그가 있는 요소만 순회
        # H1 태그 -> # 변환
        if element.name == "h1":
            markdown += f"# {element.get_text(strip=True)}\n\n"

        # H2 태그 -> ## 변환
        elif element.name == "h2":
            h2s.append(element.get_text(strip=True))
            if len(h2s) == 3:
                markdown += f"## {" ".join(h2s)}\n\n"
                h2s = []

        # p 태그 -> 리스트 항목 변환
        elif element.name == "p":
            markdown += f"- {element.get_text(strip=True)}\n"

        # strong 태그 -> bold 변환
        elif element.name == "strong":
            markdown += f"**{element.get_text(strip=True)}**"

        # div.my-callout -> callout 변환
        elif element.name == "div" and "my-callout" in element.get("class", []):
            markdown += f"> [!NOTE]\n> {element.get_text(strip=True)}\n\n"

        # blockquote -> 인용 변환
        elif element.name == "blockquote":
            markdown += f"> {element.get_text(strip=True)}\n\n"

        # a 태그 -> URL 링크 변환
        elif element.name == "a" and element.get("href"):
            link_text = element.get_text(strip=True)
            url = element["href"]
            markdown += f"[{link_text}]({url})"

        # 지정되지 않은 태그는 내용 포함하여 삭제
        else:
            continue  # 다른 태그는 무시하고 넘어감

    return markdown


# 변환 실행
markdown_content = html_to_markdown(soup)
markdown_content

"# 'AII-in 코딩 공모전’ 수상작을 소개합니다\n\n- 코딩은 더 이상 개발자만의 영역이 아닙니다. 누구나 아이디어만 있다면 창의적인 서비스를 만들어 세상을 바꿀 수 있습니다. 스파르타코딩클럽에서는 이러한 가능성을 믿고, 누구나 코딩을 통해 자신의 아이디어를 실현하고 실제 문제를 해결하는 경험을 쌓을 수 있도록 다양한 프로그램을 마련하고 있습니다.\n- <All-in> 코딩 공모전은 대학생들이 캠퍼스에서 겪은 불편함과 문제를 자신만의 아이디어로 해결해보는 대회였는데요. 이번 공모전에서 다양한 혁신적인 아이디어와 열정으로 가득한 수많은 프로젝트가 탄생했습니다. 그중 뛰어난 성과를 낸 수상작 6개를 소개합니다.\n##  🏆\xa0대상 [Lexi Note] 언어공부 필기 웹 서비스\n\n- 서비스 제작자: 다나와(김다애, 박나경)\n**서비스 제작자: 다나와(김다애, 박나경)**> [!NOTE]\n> 💡W는 어문학을 전공하는 대학생입니다. 매일 새로운 단어와 문장 구조를 공부하고 있지만, 효율적으로 학습하는 것이 쉽지 않았습니다. 단어의 의미를 찾기 위해 사전을 뒤적이고, 긴 문장을 이해하려고 번역기를 사용하다 보면, 필기 노트는 어느새 뒷전으로 밀려났거든요. 사전, 번역기, 원서, 필기노트를 왔다 갔다 하다 보면 시간이 다 지나가 버리곤 했죠.\n\n- \n- W와 같이 어문 전공생은 문법, 어휘, 문장 구조 등 다양한 자료를 학습해야 합니다. 여러 자료를 번갈아 학습하다보니 ‘사전-번역기-원서-필기노트’ 왕복으로 학습 효율이 나지 않아 고민인 경우도 많으실 거예요. <Lexi Note>는 단어를 드래그하면 네이버 사전으로 바로 연동 돼 단어의 의미를 찾으며 동시에 필기 할 수 있어요. 이외에도 번역 버튼을 누르면 파파고 번역기가 연동돼 긴 문장도 쉽게 이해할 수 있어요. 언어 학습에 필요한 할일 목록과 스케줄 템플릿을 제공하여 효율적으로 공부할 수 있습니다. 필기, 사전, 번역을 한번에 쉽고 편하게 이용할 수 있죠. 더 이상 시간 낭비 없이 효율적으로 어

In [110]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=100  # 청크 크기  # 청크 간의 중복 크기
)
split = splitter.split_text(markdown_content)
split

# Document 객체로 변환
documents = [Document(page_content=chunk) for chunk in split]

documents

[Document(metadata={}, page_content="# 'AII-in 코딩 공모전’ 수상작을 소개합니다\n\n- 코딩은 더 이상 개발자만의 영역이 아닙니다. 누구나 아이디어만 있다면 창의적인 서비스를 만들어 세상을 바꿀 수 있습니다. 스파르타코딩클럽에서는 이러한 가능성을 믿고, 누구나 코딩을 통해 자신의 아이디어를 실현하고 실제 문제를 해결하는 경험을 쌓을 수 있도록 다양한 프로그램을 마련하고 있습니다.\n- <All-in> 코딩 공모전은 대학생들이 캠퍼스에서 겪은 불편함과 문제를 자신만의 아이디어로 해결해보는 대회였는데요. 이번 공모전에서 다양한 혁신적인 아이디어와 열정으로 가득한 수많은 프로젝트가 탄생했습니다. 그중 뛰어난 성과를 낸 수상작 6개를 소개합니다.\n##  🏆\xa0대상 [Lexi Note] 언어공부 필기 웹 서비스\n\n- 서비스 제작자: 다나와(김다애, 박나경)\n**서비스 제작자: 다나와(김다애, 박나경)**> [!NOTE]\n> 💡W는 어문학을 전공하는 대학생입니다. 매일 새로운 단어와 문장 구조를 공부하고 있지만, 효율적으로 학습하는 것이 쉽지 않았습니다. 단어의 의미를 찾기 위해 사전을 뒤적이고, 긴 문장을 이해하려고 번역기를 사용하다 보면, 필기 노트는 어느새 뒷전으로 밀려났거든요. 사전, 번역기, 원서, 필기노트를 왔다 갔다 하다 보면 시간이 다 지나가 버리곤 했죠."),
 Document(metadata={}, page_content='- \n- W와 같이 어문 전공생은 문법, 어휘, 문장 구조 등 다양한 자료를 학습해야 합니다. 여러 자료를 번갈아 학습하다보니 ‘사전-번역기-원서-필기노트’ 왕복으로 학습 효율이 나지 않아 고민인 경우도 많으실 거예요. <Lexi Note>는 단어를 드래그하면 네이버 사전으로 바로 연동 돼 단어의 의미를 찾으며 동시에 필기 할 수 있어요. 이외에도 번역 버튼을 누르면 파파고 번역기가 연동돼 긴 문장도 쉽게 이해할 수 있어요. 언어 학습에 필요한 할일 목록과 스케줄 템플릿을 제공하여

In [80]:
import re
from uuid import uuid4
from langchain.schema import Document


# 고유 ID 생성 함수
def generate_id():
    return str(uuid4())


# 마크다운을 Document 객체로 변환하는 함수 (헤더 기준 분리)
def parse_markdown_to_documents(markdown):
    documents = []
    current_section = None

    for line in markdown.splitlines():
        # H1 태그 처리
        if line.startswith("# "):
            # 이전 섹션을 Document로 저장
            if current_section:
                documents.append(
                    Document(
                        page_content=current_section["content"],
                        metadata=current_section["metadata"],
                    )
                )
            # 새로운 섹션 초기화
            current_section = {
                "content": line + "\n",
                "metadata": {
                    "id": generate_id(),
                    "level": "h1",
                    "title": line[2:].strip(),
                    "editor": "신수지",
                },
            }

        # H2 태그 처리
        elif line.startswith("## "):
            # 현재 섹션이 비어있지 않으면 저장
            if current_section and current_section["content"].strip():
                documents.append(
                    Document(
                        page_content=current_section["content"],
                        metadata=current_section["metadata"],
                    )
                )
            # 새로운 섹션 초기화
            current_section = {
                "content": line + "\n",
                "metadata": {
                    "id": generate_id(),
                    "level": "h2",
                    "title": line[3:].strip(),
                    "editor": "신수지",
                },
            }

        # 본문 내용 추가
        else:
            if current_section:
                current_section["content"] += line + "\n"

    # 마지막 섹션 추가
    if current_section:
        documents.append(
            Document(
                page_content=current_section["content"],
                metadata=current_section["metadata"],
            )
        )

    return documents


# 마크다운 섹션 파싱
sections = parse_markdown_to_documents(markdown_content)
sections

[Document(metadata={'id': '37008d87-7bf0-4292-82ae-3b93c4a712f2', 'level': 'h2', 'title': '🏅 입선', 'editor': '신수지'}, page_content='## 🏅 입선\n\n'),
 Document(metadata={'id': 'da8de79d-fbe7-4730-baed-4eafed89a071', 'level': 'h2', 'title': '[BLOTIE] 교내 외국인X내국인 매칭 및 교류 플랫폼', 'editor': '신수지'}, page_content='## [BLOTIE] 교내 외국인X내국인 매칭 및 교류 플랫폼\n\n- 서비스 제작자: 블로티(이은주, 한명수, 황준영)\n')]

In [79]:
sections

[Document(metadata={'id': 'a76de647-f42e-432e-8a7a-c87d5b08f425', 'level': 'h2', 'title': '🏅 입선', 'editor': '신수지'}, page_content='## 🏅 입선\n\n'),
 Document(metadata={'id': 'cc8fa527-fc65-4d9f-b05b-59b0e11d4d87', 'level': 'h2', 'title': '[BLOTIE] 교내 외국인X내국인 매칭 및 교류 플랫폼', 'editor': '신수지'}, page_content='## [BLOTIE] 교내 외국인X내국인 매칭 및 교류 플랫폼\n\n- 서비스 제작자: 블로티(이은주, 한명수, 황준영)\n')]

In [111]:
vectorstore = FAISS.from_documents(documents=documents, embedding=OpenAIEmbeddings())

In [112]:
vectorstore.save_local("vector_stores/faiss_all-in-challenge_winner")

In [60]:
vectorstore = FAISS.load_local(
    "vector_stores/faiss_all-in-challenge_winner",
    embeddings=OpenAIEmbeddings(),
    allow_dangerous_deserialization=True,
)
retriever = vectorstore.as_retriever()

In [56]:
res = retriever.invoke("입상한 작품을 소개해줘")


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

'## 🏅\xa0입선\n\n\n\n## 🏅\xa0입선\n\n\n\n## 🏅\xa0입선\n\n\n\n## 🏆\xa0대상\n\n'